In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
from os.path import exists
import random
import rtsvg
rt = rtsvg.RACETrack()
relates    = [('fm','to')]
_filename_layouts_ = '../rtsvg/config/simple_graph_layouts.csv'
_filename_df_      = '../rtsvg/config/simple_graphs_df.csv'
def randStr(n=5): return ''.join(random.choices('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ', k=n))

In [2]:
# subgraph extraction example code
#g = rt.createNetworkXGraph(pl.DataFrame(_lu_), relates)
#for _comp_ in nx.connected_components(g):
#    g.subgraph(_comp_)

In [3]:
#
# graphCreateAllEdgeCombinationsForConnectedGraph()
# - This function will create all possible connected graphs with the given number of nodes
#
# Scalability:
# - 5 nodes -- time taken is  0.5s on M1 Pro
# - 6 nodes -- time taken is 24.6s on M1 Pro
#
def graphCreateAllEdgeCombinationsForConnectedGraph(nodes=6):
    if nodes > 6: raise ValueError('Too many nodes -- only up to 6 nodes supported')
    max_edges           = nodes*(nodes-1)//2
    gs_found, lus_found = [], []
    for i in range(1, 2**max_edges):
        mini_lu = {'fm':[], 'to':[]}
        bit_no  = 0
        for node_0 in range(nodes):
            for node_1 in range(node_0+1, nodes):
                if i & (1<<bit_no): mini_lu['fm'].append(f'{node_0}'), mini_lu['to'].append(f'{node_1}')
                bit_no += 1
        # Create this spectific graph
        g_mini = rt.createNetworkXGraph(pl.DataFrame(mini_lu), relates)
        # Make sure it is connected and has the right number of nodes
        if nx.is_connected(g_mini) and nx.number_of_nodes(g_mini) == nodes:
            # Make sure this graph is not already found
            already_seen = False
            for _other_ in gs_found:
                if nx.is_isomorphic(g_mini, _other_):
                    already_seen = True
                    break
            # If it is not already found, add it
            if not already_seen:
                gs_found.append(g_mini)
                lus_found.append(mini_lu)
    # transpose into a single dictionary
    _lu_ = {'fm':[], 'to':[], 'fm_i':[], 'to_i':[], 'graph_no':[]}
    for graph_no in range(len(lus_found)):
        graph_no_str = f'{nodes}_{graph_no}'
        my_lu = lus_found[graph_no]
        for i in range(len(my_lu['fm'])):
            _lu_['fm']      .append(f'{graph_no_str}_{my_lu["fm"][i]}')
            _lu_['to']      .append(f'{graph_no_str}_{my_lu["to"][i]}')
            _lu_['fm_i']    .append(int(my_lu['fm'][i]))
            _lu_['to_i']    .append(int(my_lu['to'][i]))
            _lu_['graph_no'].append(graph_no_str)
    return _lu_

if exists(_filename_df_): 
    df = pl.read_csv(_filename_df_)
else:
    _dfs_ = []
    for num_of_nodes in range(2,7): _dfs_.append(pl.DataFrame(graphCreateAllEdgeCombinationsForConnectedGraph(num_of_nodes)))
    df = pl.concat(_dfs_)
    df.to_csv(_filename_df_)

In [ ]:
pos = {}
if exists(_filename_layouts_):
    df_pos = pd.read_csv(_filename_layouts_)
    for i in range(len(df_pos)): pos[df_pos['node'][i]] = (df_pos['x'][i], df_pos['y'][i])
igl = rt.interactiveGraphLayout(df, ln_params={'relationships':relates, 'pos':pos}, w=1600, h=900)
igl

In [5]:
lu_save = {'node':[], 'x':[], 'y':[]}
for _node_ in igl.pos:
    lu_save['node'].append(_node_)
    lu_save['x']   .append(igl.pos[_node_][0])
    lu_save['y']   .append(igl.pos[_node_][1])
pd.DataFrame(lu_save).to_csv(_filename_layouts_, index=False)